In [1]:
import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
from IPython.display import clear_output
%matplotlib inline

In [2]:
root_path="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/database"

In [3]:
def mfcc_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [4]:
#NOT USED
def fft_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    fft = np.fft.fft(signal)
    
    return fft

In [5]:
#NOT USED
signal, sample_rate = librosa.load('database/ac/ac_AHCG_FSRPNCW.wav')
ipd.Audio('database/ac/ac_AHCG_FSRPNCW.wav')
fft = np.fft.fft(signal)
fft

array([ 3.96959355+0.j        , -1.57368256-0.43014124j,
       -0.5562213 +0.51962332j, ...,  0.00995858+0.12657352j,
       -0.5562213 -0.51962332j, -1.57368256+0.43014124j])

In [6]:
#NOT USED
signal, sample_rate = librosa.load('database\sol\sol_AKZI_FAWCVUT.wav' )
ipd.Audio('database\sol\sol_AKZI_FAWCVUT.wav' )
mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
mfccs_features.shape

(40, 44)

In [7]:
"""from tqdm import tqdm
import time"""
extracted_feataures =[]
for root, directories, files in os.walk(root_path):
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names))
            data = mfcc_extractor(os.path.join(root,file_names))
            extracted_feataures.append([data,os.path.basename(os.path.normpath(root))])

    """for i in tqdm (range (100), desc="Loading..."):
        time.sleep(3)"""

In [8]:
extracted_feataures_dataframe= pd.DataFrame(extracted_feataures, columns=["mfcc", "word"])
extracted_feataures_dataframe.head()

,mfcc,word
0,"[-483.0338, 62.193104, -10.424236, 17.89255, -...",ac
1,"[-393.0876, 61.467396, -20.944004, 18.326431, ...",ac
2,"[-557.86304, 99.75818, -6.094904, 0.65864086, ...",ac
3,"[-518.9073, 52.873825, -3.625966, 5.807297, -9...",ac
4,"[-539.5348, 84.21058, -6.7271714, 4.8634925, -...",ac


In [9]:
X=np.array(extracted_feataures_dataframe['mfcc'].tolist())
Y=np.array(extracted_feataures_dataframe['word'].tolist())

In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
Y_numerical=to_categorical(labelencoder.fit_transform(Y))

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y_numerical, test_size=0.2, random_state=0)

In [12]:
#NOT USED
X_train.shape

(21153, 40)

In [13]:
#NOT USED
len(Y_test)

5289

ANN Model

In [72]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [73]:
def createmodel():
    model= Sequential()

    model.add(Flatten(input_shape=(40, )))
    model.add(Dense(300, Activation('relu')))
    model.add(Dropout(0.2))
    model.add(Dense(300, Activation('relu')))
    model.add(Dense(Y_numerical.shape[1], Activation('softmax')))

    return model

model=createmodel()
model

"""
model.add(Dense(200, input_shape=(40,)))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(Y_numerical.shape[1]))
model.add(Activation('sigmoid'))"""

"\nmodel.add(Dense(200, input_shape=(40,)))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(300))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(50))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(Y_numerical.shape[1]))\nmodel.add(Activation('sigmoid'))"

In [74]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 40)                0         
                                                                 
 dense_18 (Dense)            (None, 300)               12300     
                                                                 
 dropout_6 (Dropout)         (None, 300)               0         
                                                                 
 dense_19 (Dense)            (None, 300)               90300     
                                                                 
 dense_20 (Dense)            (None, 14)                4214      
                                                                 
Total params: 106,814
Trainable params: 106,814
Non-trainable params: 0
_________________________________________________________________


In [75]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [76]:
#checkpointpath="C:/Users/sarib/Desktop/Gohm/saved_models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpointpath="training3/cp.ckpt"
checkpointer = ModelCheckpoint(checkpointpath, monitor='val_accuracy',
                               verbose=2, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
#log_csv = CSVLogger('csv_logs.csv', separator=',', append=False)
#callbackList= [checkpointer, early_stop, log_csv]
callbackList= [checkpointer, early_stop]
#model.fit(X_train, Y_train, epochs=70, validation_data=(X_test, Y_test), callbacks=[checkpointer], verbose=2)
model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), callbacks=callbackList, verbose=2)
#model.save('pretrained_model.h5')



Epoch 1/200

Epoch 1: val_accuracy improved from -inf to 0.49026, saving model to training3\cp.ckpt


INFO:tensorflow:Assets written to: training3\cp.ckpt\assets


INFO:tensorflow:Assets written to: training3\cp.ckpt\assets


662/662 - 3s - loss: 3.9532 - accuracy: 0.2879 - val_loss: 1.5166 - val_accuracy: 0.4903 - 3s/epoch - 5ms/step
Epoch 2/200

Epoch 2: val_accuracy improved from 0.49026 to 0.51484, saving model to training3\cp.ckpt


INFO:tensorflow:Assets written to: training3\cp.ckpt\assets


...

INFO:tensorflow:Assets written to: training3\cp.ckpt\assets


662/662 - 3s - loss: 0.5847 - accuracy: 0.7926 - val_loss: 0.8066 - val_accuracy: 0.7502 - 3s/epoch - 4ms/step
Epoch 41/200

Epoch 41: val_accuracy did not improve from 0.75024
662/662 - 2s - loss: 0.5835 - accuracy: 0.7938 - val_loss: 0.8334 - val_accuracy: 0.7412 - 2s/epoch - 3ms/step
Epoch 42/200

Epoch 42: val_accuracy did not improve from 0.75024
662/662 - 2s - loss: 0.5746 - accuracy: 0.7947 - val_loss: 0.8722 - val_accuracy: 0.7317 - 2s/epoch - 3ms/step
Epoch 43/200

Epoch 43: val_accuracy did not improve from 0.75024
662/662 - 2s - loss: 0.5747 - accuracy: 0.7988 - val_loss: 0.8261 - val_accuracy: 0.7366 - 2s/epoch - 3ms/step
Epoch 44/200

Epoch 44: val_accuracy did not improve from 0.75024
662/662 - 2s - loss: 0.5749 - accuracy: 0.7937 - val_loss: 0.8334 - val_accuracy: 0.7457 - 2s/epoch - 3ms/step
Epoch 45/200

Epoch 45: val_accuracy did not improve from 0.75024
662/662 - 2s - loss: 0.5690 - accuracy: 0.7985 - val_loss: 0.8240 - val_accuracy: 0.7451 - 2s/epoch - 3ms/step
Epoc

INFO:tensorflow:Assets written to: training3\cp.ckpt\assets


INFO:tensorflow:Assets written to: training3\cp.ckpt\assets


662/662 - 3s - loss: 0.5610 - accuracy: 0.8001 - val_loss: 0.8258 - val_accuracy: 0.7523 - 3s/epoch - 4ms/step
Epoch 47/200

Epoch 47: val_accuracy did not improve from 0.75232
662/662 - 2s - loss: 0.5535 - accuracy: 0.8011 - val_loss: 0.8339 - val_accuracy: 0.7485 - 2s/epoch - 3ms/step
Epoch 48/200

Epoch 48: val_accuracy did not improve from 0.75232
662/662 - 2s - loss: 0.5471 - accuracy: 0.8071 - val_loss: 0.8336 - val_accuracy: 0.7514 - 2s/epoch - 3ms/step
Epoch 49/200

Epoch 49: val_accuracy did not improve from 0.75232
662/662 - 2s - loss: 0.5431 - accuracy: 0.8096 - val_loss: 0.8439 - val_accuracy: 0.7459 - 2s/epoch - 3ms/step
Epoch 50/200

Epoch 50: val_accuracy did not improve from 0.75232
662/662 - 2s - loss: 0.5323 - accuracy: 0.8101 - val_loss: 0.8492 - val_accuracy: 0.7413 - 2s/epoch - 3ms/step
Epoch 50: early stopping


In [77]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 0s 3ms/step - loss: 0.8492 - accuracy: 0.7413
Test Score:  [0.8491846323013306, 0.741349995136261]
662/662 [==============================] - 2s 2ms/step - loss: 0.2955 - accuracy: 0.9019
Train Score:  [0.2955106198787689, 0.9019051790237427]


In [78]:
model.load_weights(checkpointpath)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [79]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 1s 2ms/step - loss: 0.8258 - accuracy: 0.7523
Test Score:  [0.8257700800895691, 0.7523161172866821]
662/662 [==============================] - 1s 2ms/step - loss: 0.2882 - accuracy: 0.9040
Train Score:  [0.2881965637207031, 0.9039852619171143]


In [ ]:
"""plt.plot(model.history['loss'], label='train')
plt.plot(model.history['val_loss'], label='test')
plt.legend()
plt.show()"""

"plt.plot(model.history['loss'], label='train')\nplt.plot(model.history['val_loss'], label='test')\nplt.legend()\nplt.show()"

In [ ]:
#model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

PREDICTION WITH DATA

In [80]:
predictiondata_feataures =[]
counter_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/RecordsFromDataset"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicted_label=np.argmax(model.predict(rl), axis=-1)
            prediction_class = labelencoder.inverse_transform(predicted_label) 
            predictiondata_feataures.append([os.path.basename(os.path.normpath(root)), prediction_class, predicted_label])
            if os.path.basename(os.path.normpath(root))==prediction_class:
                counter_true+=1

In [81]:
predictiondata_feataures_dataframe= pd.DataFrame(predictiondata_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentage= (counter_true/len(predictiondata_feataures))*100
print("The percantage of the true predicted data: ", percentage,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondata_feataures_dataframe


The percantage of the true predicted data:  66.66666666666666 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[dur],[4]
2,ac,[iptal],[9]
3,asagi,[asagi],[1]
4,asagi,[hayir],[7]
5,asagi,[baslat],[2]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[baslat],[2]
9,devam,[devam],[3]


In [82]:
predictiondatas_feataures =[]
counters_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/Records"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicteds_label=np.argmax(model.predict(rl), axis=-1)
            predictions_class = labelencoder.inverse_transform(predicteds_label) 
            predictiondatas_feataures.append([os.path.basename(os.path.normpath(root)), predictions_class, predicteds_label])
            if os.path.basename(os.path.normpath(root))==predictions_class:
                counters_true+=1

In [83]:
predictiondatas_feataures_dataframe= pd.DataFrame(predictiondatas_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentages= (counters_true/len(predictiondatas_feataures))*100
print("The percantage of the true predicted data: ", percentages,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondatas_feataures_dataframe


The percantage of the true predicted data:  52.38095238095239 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[ac],[0]
2,ac,[asagi],[1]
3,asagi,[asagi],[1]
4,asagi,[asagi],[1]
5,asagi,[asagi],[1]
6,baslat,[asagi],[1]
7,baslat,[baslat],[2]
8,baslat,[baslat],[2]
9,devam,[devam],[3]


In [ ]:
"""filename="predictionData/dursultan.wav"
r=mfcc_extractor(filename)
print(r)
rl=r.reshape(1,-1)
print(rl)
print(rl.shape)
predicted_label=np.argmax(model.predict(rl), axis=-1)

print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class"""

'filename="predictionData/dursultan.wav"\nr=mfcc_extractor(filename)\nprint(r)\nrl=r.reshape(1,-1)\nprint(rl)\nprint(rl.shape)\npredicted_label=np.argmax(model.predict(rl), axis=-1)\n\nprint(predicted_label)\nprediction_class = labelencoder.inverse_transform(predicted_label) \nprediction_class'